# Checking the usability of actimetric data
Author: Marion Granier  
Date: 2025-03-01  
  
This script uses the Actipy library to analyze accelerometer data.  
Check prerequisites before running the script.  
Source: https://github.com/OxWearables/actipy/blob/master/README.md

In [ ]:
# Importing the required libraries
import actipy
import gzip
import shutil
import pandas as pd

## For a single actimetric data file

In [ ]:
# Data information
patient_ID = "C1P20" # CXPXX
month_ID = "1" # "0" = first month ; "1" = second month ; "2" = third month ; "3" = fourth month ; "4" = fifth month ; "5" = sixth month
side = "left" # "left" or "right"

# File paths
input_path = f"../data/data_actimetry/{patient_ID}_M{month_ID}/{side}.cwa"
output_path = f"../data/data_actimetry/{patient_ID}_M{month_ID}/{side}.cwa.gz"

In [ ]:
# Compression of the .cwa file to .gz
with open(input_path, "rb") as f_in, gzip.open(output_path, "wb") as f_out:
    shutil.copyfileobj(f_in, f_out)

# Read the data and detect non-wearing periods
data, info = actipy.read_device(output_path,
                                 lowpass_hz=20,
                                 calibrate_gravity=False,
                                 detect_nonwear=True,
                                 resample_hz=10)

info

# Collect the time of wear variable
wear_time_day = info["WearTime(days)"]

# Check if the data is exploitable
if wear_time_day < 5:
    print(f"***** Data not exploitable for {patient_ID}, M{month_ID}, {side} side: {wear_time_day} days of wearing *****")
else:
    print(f"Data exploitable for {patient_ID}, M{month_ID}, {side} side: {wear_time_day} days of wearing")

## For all actimetric data files

In [ ]:
# List of patient IDs, month IDs and sides
participants_info = pd.read_csv('../data/participants_1.csv', sep=';')
patient_IDs = participants_info['folder_name'].str.split('_M').str[0].tolist()
print(patient_IDs)
month_IDs = ["0", "1", "2", "3", "4", "5"]
sides = ["left", "right"]

# Process each patient
for patient_ID in patient_IDs:
    months_inexploitable = 0
    
    for month_ID in month_IDs:
        for side in sides:
            input_path = f"../data/data_actimetry/{patient_ID}_M{month_ID}/{side}.cwa"
            output_path = f"../data/data_actimetry/{patient_ID}_M{month_ID}/{side}.cwa.gz"
            
            try:
                # Compression of the .cwa file to .gz
                with open(input_path, "rb") as f_in, gzip.open(output_path, "wb") as f_out:
                    shutil.copyfileobj(f_in, f_out)
                
                # Read the data and detect non-wearing periods
                data, info = actipy.read_device(output_path,
                                                lowpass_hz=20,
                                                calibrate_gravity=False,
                                                detect_nonwear=True,
                                                resample_hz=10)
                
                # Collect the time of wear variable
                wear_time_day = info["WearTime(days)"]
                
                # Check if the data is exploitable
                if wear_time_day < 5:
                    print(f"***** Data not exploitable for {patient_ID}, M{month_ID}, {side} side: {wear_time_day} days of wearing *****")
                    months_inexploitable += 1
#                else:
#                    print(f"Data exploitable for {patient_ID}, M{month_ID}, {side} side: {wear_time_day} days of wearing")
            
            except FileNotFoundError:
                print(f"***** File not found: {input_path} *****")
                months_inexploitable += 1
            except Exception as e:
                print(f"*****Error processing {input_path}: {e} *****")
    
    if months_inexploitable > 2:
        print(f"***** Patient {patient_ID} is not exploitable *****")
    else:
        print(f"Patient {patient_ID} is exploitable")


['C1P20', 'C1P30', 'C1P31', 'C1P32', 'C1P33']
***** File not found: ../data/data_actimetry/C1P20_M0/left.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M0/right.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M1/left.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M1/right.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M2/left.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M2/right.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M3/left.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M3/right.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M4/left.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M4/right.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M5/left.cwa *****
***** File not found: ../data/data_actimetry/C1P20_M5/right.cwa *****
***** Patient C1P20 is not exploitable *****
***** File not found: ../data/data_actimetry/C1P30_M0/left.cwa *****
***** 